# Mnist classification demo

In [3]:
import grpc 
import numpy as np
import hydro_serving_grpc as hs
from grpc import ssl_channel_credentials
import keras
from keras.datasets import mnist
from keras import backend as K

## Read data sample

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_rows, img_cols = 28, 28
num_classes = 10

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Setup server and model

In [51]:

channel = grpc.secure_channel("dev.k8s.hydrosphere.io", credentials=ssl_channel_credentials())
stub = hs.PredictionServiceStub(channel) 
model_spec = hs.ModelSpec(name="new_mnist", signature_name="INFERRED_PIPELINE_SIGNATURE")

## Create tensors

In [52]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=1), hs.TensorShapeProto.Dim(size=28), hs.TensorShapeProto.Dim(size=28), hs.TensorShapeProto.Dim(size=1)])
tensor = hs.TensorProto(dtype=hs.DT_FLOAT, tensor_shape=tensor_shape, float_val=np.array([x_test[0]]).flatten().astype(np.float32))


## Send request to model servable

In [53]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"imgs": tensor}) 
result = stub.Predict(request)
print('Output values:')
print(result)
print('Ground truth label:')
print(y_test[0])

Output values:
outputs {
  key: "class_ids"
  value {
    dtype: DT_INT64
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
    int64_val: 7
  }
}
outputs {
  key: "classes"
  value {
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
    string_val: "7"
  }
}
outputs {
  key: "logits"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 10
      }
    }
    float_val: -5.008371353149414
    float_val: 0.8771364092826843
    float_val: 2.5406460762023926
    float_val: 5.5690765380859375
    float_val: -6.439862251281738
    float_val: -3.771909475326538
    float_val: -10.568458557128906
    float_val: 15.584836959838867
    float_val: -3.4896159172058105
    float_val: 7.514048099517822
  }
}
outputs {
  key: "probabilities"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
  